In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# config.gpu_options.allow_growth = True
# set_session(tf.Session(config=config)) 

## LIMIT GPU USAGE
config = tf.ConfigProto(log_device_placement=True)  
config.gpu_options.allow_growth = True  # don't pre-allocate memory; allocate as-needed
config.gpu_options.per_process_gpu_memory_fraction = 0.95  # limit memory to be allocated
set_session(tf.Session(config=config)) # create sess w/ above settings
print(tf.test.is_built_with_cuda())

Using TensorFlow backend.


True


In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [3]:
# keras example imports
from keras.models import load_model
 
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

k.get_session().close()
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.95
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))
###################################
print(tf.test.is_built_with_cuda())

True


In [4]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import os
import numpy as np
from tabulate import tabulate
from statistics import mean

In [5]:
#this function is to get the time string like h:m:s
#========================================================================================
def getTime(time):
    time=time%(24*3600)
    hours=time//3600
    time%=3600
    minutes=time//60
    time%=60
    seconds=time
    periods=[('hours',int(hours)),('minutes',int(minutes)),('seconds',int(seconds))]
    time_string=':'.join('{}'.format(value) for name,value in periods)
    return time_string
#========================================================================================

In [6]:
import csv
dirpath = os.getcwd()
path = dirpath + '/LSTMExperimentResults_AfterDefense/1%/RMSE_Confidence_Interval (epoch= 10, batch = 20 , neurons = 10).csv'
myfile1 = open(path,'w', newline='')
writer1 = csv.writer(myfile1)
heading =['Samples','Number of Observations','RMSEValues','Mean','Standard Errors','Upper Bound','Lower Bound','Execution Time']
writer1.writerow(heading)
myfile1.close()

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/
https://machinelearningmastery.com/models-sequence-prediction-recurrent-neural-networks/
https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

In [7]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# specify the number of lag hours
list_n_mins = [1,5,10,15,30,60]

#percentage = ['5%','10%','15%','20%','25%','30%','35%','40%','45%','50%']
percentage = ['1%']


for n_mins in list_n_mins:
    
    for percent in percentage:
        
        import time
        start_time = time.time()

        # load dataset
        dirpath = os.getcwd()
        dataset = read_csv(dirpath + '/datasetForLSTM/60_'+percent+'_Allcombine.csv', header=0, index_col=0)
        values = dataset.values

        epochs = 10
        batch_size = 20
        neurons = 10

        n_features = 6
        # frame as supervised learning
        reframed = series_to_supervised(values, n_mins, 1)
        print(reframed.head())

        # drop columns we don't want to predict
        # for number of mins  = 1, drop columns ==> [6,7,8,9,10] , index 11 for gridlock
        # for number of mins  = 5, drop columns ==> [30,31,32,33,34] , index 35 for gridlock
        # for number of mins  = 10, drop columns ==> [60,61,62,63,64] , index 65 for gridlock
        # for number of mins  = 15, drop columns ==> [90,91,92,93,94] , index 95 for gridlock
        # for number of mins  = 30, drop columns ==> [180,181,182,183,184] , index 95 for gridlock
        # for number of mins  = 60, drop columns ==> [360,361,362,363,364] , index 365 for gridlock
        # for number of mins  = 90, drop columns ==> [540,541,542,543,544] , index 545 for gridlock
        
        if n_mins == 1:
            reframed.drop(reframed.columns[[6,7,8,9,10]], axis=1, inplace=True)
        if n_mins == 5:
            reframed.drop(reframed.columns[[30,31,32,33,34]], axis=1, inplace=True)
        if n_mins == 10:
            reframed.drop(reframed.columns[[60,61,62,63,64]], axis=1, inplace=True)
        if n_mins == 15:
            reframed.drop(reframed.columns[[90,91,92,93,94]], axis=1, inplace=True)
        if n_mins == 30:
            reframed.drop(reframed.columns[[180,181,182,183,184]], axis=1, inplace=True)
        if n_mins == 60:
            reframed.drop(reframed.columns[[360,361,362,363,364]], axis=1, inplace=True)
            
        # reframed.to_csv(dirpath + '/datasetForLSTM/60_5%_Allcombine_reframed.csv',index=False )
        #print(reframed.head())



        reframed.columns

        # split into train and test sets
        values = reframed.values
        n_train_mins =80 * 181
        train = values[:n_train_mins, :]
        test = values[n_train_mins:, :]

        # split into input and outputs
        n_obs = n_mins * n_features
        train_X, train_y = train[:, :n_obs], train[:, -1]
        test_X, test_y = test[:, :n_obs], test[:, -1]

        print(train_X.shape, len(train_X), train_y.shape)
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], n_mins, n_features))
        test_X = test_X.reshape((test_X.shape[0], n_mins, n_features))
        print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

        # design network
        model = Sequential()
        model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        # fit network
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X, test_y), verbose=0, shuffle=False)
        # plot history
        pyplot.plot(history.history['loss'], label='train')
        pyplot.plot(history.history['val_loss'], label='test')
        pyplot.xlabel('epoch')
        pyplot.ylabel('loss')
        pyplot.legend()
        pyplot.savefig(dirpath + '/LSTMExperimentResults_AfterDefense/1%/'+percent+'_'+str(n_mins)+'min_'+str(epochs)+'epochs_'+str(batch_size)+'batch_size_'+str(neurons)+'neurons.png')
        pyplot.clf()
        # pyplot.show()


        # make a prediction
        yhat = model.predict(test_X)

#         temp_yhat = yhat 
#         temp_yhat = [np.round(num) for num in yhat]
#         pyplot.plot(test_y,  'r-',label='actual')
#         pyplot.show()
#         pyplot.plot(temp_yhat, 'b-',label='predict')
#         pyplot.show()

        test_X.shape

        repeats  = 10
        rmse_list = list()
        
        acutal_predicted_df = DataFrame()
        acutal_predicted_df['actual']= test_y
        acutal_predicted_df['predicted']= yhat
        acutal_predicted_df.to_csv(dirpath + '/LSTMExperimentResults_AfterDefense/1%/'+percent+'_'+str(n_mins)+'min_'+str(epochs)+'epochs_'+str(batch_size)+'batch_size_'+str(neurons)+'neurons.csv', index=False)


        for r in range(repeats):

            # make a prediction
            test_X, test_y = test[:, :n_obs], test[:, -1]
            test_X = test_X.reshape((test_X.shape[0], n_mins, n_features))
            yhat = model.predict(test_X)
            yhat.shape


            test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]*test_X.shape[2]))
            test_y = test_y.reshape((len(test_y), 1))

            # invert scaling for forecast
            inv_yhat = concatenate((yhat, test_X), axis=1)


            #print(tabulate(inv_yhat, headers=['inv_yhat'], tablefmt='orgtbl'))

            inv_y = concatenate((test_y, test_X), axis=1)
            #print(tabulate(inv_y, headers=['inv_y'], tablefmt='orgtbl'))

            # calculate RMSE
            rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
            rmse_list.append(rmse)

        import numpy as np
        import scipy.stats
        import csv


        a = 1.0 * np.array(rmse_list)
        n = len(a)
        mean, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + 0.95) / 2., n-1)


        elapsed_time = getTime(time.time() - start_time)

        myfile = open(dirpath + '/LSTMExperimentResults_AfterDefense/1%/RMSE_Confidence_Interval (epoch= 10, batch = 20 , neurons = 10).csv', 'a', newline='')
        writer = csv.writer(myfile)
        with myfile:
            writer.writerow(
                [percent,n_mins , rmse_list,mean, se, mean-h, mean+h,elapsed_time])    

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  var1(t)  \
1        0.0        0.0        0.0        0.0        0.0        0.0        0   
2        0.0        0.0        0.0        0.0        0.0        0.0        0   
3        0.0        0.0        0.0        0.0        0.0        0.0        0   
4        0.0        0.0        0.0        0.0        0.0        0.0        0   
5        0.0        0.0        0.0        0.0        0.0        0.0        0   

   var2(t)  var3(t)  var4(t)  var5(t)  var6(t)  
1        0        0        0        0        0  
2        0        0        0        0        0  
3        0        0        0        0        0  
4        0        0        0        0        0  
5        0        0        0        0        0  
(14480, 6) 14480 (14480,)
(14480, 1, 6) (14480,) (3619, 1, 6) (3619,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
   var1(t-5)  var2(t-5)  var3(t-5) 

<Figure size 432x288 with 0 Axes>